In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest,f_classif
data = pd.read_csv("housing.csv")
data.columns
data.head()
data.describe()
data.info()
data.isnull().sum()
data.total_bedrooms.unique()
data.total_bedrooms.fillna(data['total_bedrooms'].mean(),inplace=True)
data.isnull().sum()
import plotly.express as px
px.box(data.select_dtypes(include="float64"))
data.columns
import numpy as np
numcol = ['total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value']

for col in numcol:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    print("column :",col)
    IQR = Q3 - Q1
    print(Q1,Q3,IQR)
    # Define bounds
    lbound = Q1 - 1.5 * IQR
    ubound = Q3 + 1.5 * IQR
    print(lbound,ubound)
    # Replace outliers with the respective bounds
    data[col] = np.where(data[col] > ubound, ubound, 
                            np.where(data[col] < lbound, lbound, data[col]))
px.box(data.select_dtypes(include="float64"))

le = LabelEncoder()
data["ocean_proximity"] = le.fit_transform(data["ocean_proximity"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


C:\Users\venka\AppData\Local\Temp\ipykernel_6296\1928910693.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.total_bedrooms.fillna(data['total_bedrooms'].mean(),inplace=True)


column : total_rooms
1447.75 3148.0 1700.25
-1102.625 5698.375
column : total_bedrooms
297.0 643.25 346.25
-222.375 1162.625
column : population
787.0 1725.0 938.0
-620.0 3132.0
column : households
280.0 605.0 325.0
-207.5 1092.5
column : median_income
2.5633999999999997 4.74325 2.17985
-0.7063750000000004 8.013024999999999
column : median_house_value
119600.0 264725.0 145125.0
-98087.5 482412.5


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming 'data' is your DataFrame
scaler = StandardScaler()
Y = data['median_house_value']
X = scaler.fit_transform(data.drop(columns=['median_house_value']))
X = pd.DataFrame(data=X, columns=data.drop(columns=['median_house_value']).columns)

# Define the range of k values you want to test
k_values = range(1, 11)  # For example, test k from 1 to 10

# Initialize a dictionary to store the results
results = {}

for k in k_values:
    # Select the top k features
    selector = SelectKBest(score_func=f_classif, k=k)
    select1 = selector.fit_transform(X, Y)
    seleind = np.argsort(selector.scores_)[::-1][:k]
    selectfeatures = X.columns[seleind]
    
    # Print the selected features
    print(f"Top {k} features: {selectfeatures}")
    
    # Split the data into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(select1, Y, test_size=0.2, random_state=42)
    
    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, Y_train)
    
    # Make predictions
    Y_pred = model.predict(X_test)
    
    # Calculate the mean squared error
    mse = mean_squared_error(Y_test, Y_pred)
    
    # Store the results
    results[k] = mse
    
    # Print the mean squared error
    print(f"Mean Squared Error for k={k}: {mse}")

# Print the final results
print("Final Results:")
for k, mse in results.items():
    print(f"k={k}, Mean Squared Error: {mse}")

Top 1 features: Index(['median_income'], dtype='object')
Mean Squared Error for k=1: 6845933170.722345
Top 2 features: Index(['median_income', 'total_rooms'], dtype='object')
Mean Squared Error for k=2: 6845957298.62605
Top 3 features: Index(['median_income', 'total_rooms', 'population'], dtype='object')
Mean Squared Error for k=3: 6800044685.17881
Top 4 features: Index(['median_income', 'total_rooms', 'population', 'latitude'], dtype='object')
Mean Squared Error for k=4: 6664575874.755774
Top 5 features: Index(['median_income', 'total_rooms', 'population', 'latitude', 'households'], dtype='object')
Mean Squared Error for k=5: 5717327158.507097
Top 6 features: Index(['median_income', 'total_rooms', 'population', 'latitude', 'households',
       'ocean_proximity'],
      dtype='object')
Mean Squared Error for k=6: 5608484267.5869875
Top 7 features: Index(['median_income', 'total_rooms', 'population', 'latitude', 'households',
       'ocean_proximity', 'total_bedrooms'],
      dtype='obj

c:\Users\venka\AppData\Local\anaconda3\envs\ML\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning:

k=10 is greater than n_features=9. All the features will be returned.



Top 10 features: Index(['median_income', 'total_rooms', 'population', 'latitude', 'households',
       'ocean_proximity', 'total_bedrooms', 'housing_median_age', 'longitude'],
      dtype='object')
Mean Squared Error for k=10: 4813228974.413971
Final Results:
k=1, Mean Squared Error: 6845933170.722345
k=2, Mean Squared Error: 6845957298.62605
k=3, Mean Squared Error: 6800044685.17881
k=4, Mean Squared Error: 6664575874.755774
k=5, Mean Squared Error: 5717327158.507097
k=6, Mean Squared Error: 5608484267.5869875
k=7, Mean Squared Error: 5710063962.464406
k=8, Mean Squared Error: 5371956701.620215
k=9, Mean Squared Error: 4813228974.413971
k=10, Mean Squared Error: 4813228974.413971
